In [ ]:
%pip install folium

In [56]:
# Settings

# Aligns overlay with real map

xscale: float = 1
yscale: float = 101/128

yoffset: float = 12.6
xoffset: float = 10.7

# Show overlay/real map/Logo/Borders

real_map: bool = False
overlay: bool = True
logo: bool = False
borders: bool = False

background_color: str = '#2e3192'

# Zoom settings

max_zoom: int = 9

lon_bounds:int = 300
lat_bounds:int = 250

# Debugging and Settings

debug: bool = False

filename: str = "hospitalmap"

In [57]:
import folium
from folium.raster_layers import ImageOverlay

import numpy as np

import requests

    # Image bounds in latitude and longitude, [0] is the bottom right corner, [1] is the top left corner

image_bounds = [[-90*yscale + yoffset,180*xscale + xoffset], [90*yscale + yoffset,-180*xscale + xoffset]] 

# Create map, for zoom, drag, and real cords
m = folium.Map(
    location=[0,0], # Where map is initally centered
    # tiles=None,  # No basemap tiles, messes with zoom, remove map in post
    zoom_start=2, 
    max_zoom = 18 if debug else max_zoom, 
    min_zoom= 0 if debug else 2,
    zoom_control=True, 
    scrollWheelZoom=True,
    dragging=True, 
    max_bounds=True,
    max_lat=lat_bounds, # Prevent the map from being dragged too far, 300 is arbitrary but works well
    min_lat=-lat_bounds,
    max_lon=lon_bounds,
    min_lon=-lon_bounds,
)

import json

with open("hospitals.json") as f:

    data = json.load(f)

    for hospital in data["hospitals"]:

        folium.Marker(
            location=hospital["coords"],
            popup =
            
            f"""

            <h3 style="font:bold;">{hospital["name"]}</h3>
            <p>{hospital["address"]}</p>
            <p style="color:grey;">{hospital["coords"]}</p>
            
            """
            
            ,
            icon=folium.Icon(icon="hospital", prefix="fa", color="red"),
            color="blue"
        ).add_to(m)

# Overlay image, 4k image from accurate svg map

image_path = 'Overlay.png' 

# Overlay the PNG image on the map

if overlay:

    ImageOverlay(
        image=image_path,
        bounds=image_bounds,
        opacity=1
    ).add_to(m)

# JClinic Logo

if logo:

    # w: 1920, h: 762.5425

    wh_ratio: float = 1920 / 762.5425

    print(wh_ratio, -180+(3*wh_ratio), sep="\n")

    ImageOverlay(
        image="jclinic-logo.png",
        bounds=[[80,-180], [77,-180+20]],
        opacity=1
    ).add_to(m)

if borders:

    geojson_data = requests.get(
        "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/world_countries.json"
    ).json()

    folium.GeoJson(geojson_data, name="hello world", color="green", opacity=0.2).add_to(m)

# Save the map as an HTML file

m.save(f'{filename}.html')

# Edit to remove map, only show overlay and set bg to color .leaflet-container { font-size: 1rem; } -> .leaflet-tile { visibility: hidden; } .leaflet-container { font-size: 1rem; background: {background_color};}

import os

if not real_map:

    with open(f'{filename}.html', "r+") as f:

        lines: np.array = np.array(f.read().splitlines())

        index_array = np.where(np.char.find(lines, ".leaflet-container { font-size: 1rem; }") > 0)[0][0] # should only be one line

        lines[index_array] = f'.leaflet-tile {{ visibility: hidden; }} .leaflet-container {{ font-size: 1rem; background: {background_color};}}'

        # Removes files contents

        f.seek(0)

        f.write("\n".join(lines))
        
        f.truncate()

In [78]:
# Post render


# .leaflet-tile { visibility: hidden; } .leaflet-container { font-size: 1rem; background: #2E3192;}